In [1]:
from pymongo import MongoClient
import json
from tqdm import tqdm

Load data into MongoDB

In [30]:
from pymongo import MongoClient
from pathlib import Path
import os
import json
from tqdm import tqdm

# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client['RAG']
collection = db['PubMedEmbedded']

# Define the path for the source directory
source_directory = Path('C:/Users/linus/big_data/pubmed/first100JSONLembedded/')

# Check if the source directory exists
if not source_directory.exists():
    print("The source directory does not exist.")
else:
    # Iterate through each file in the source directory
    for file_name in tqdm(os.listdir(source_directory)):
        if file_name.endswith('.jsonl'):
            source_file = source_directory / file_name
            
            # Create a list to store the documents
            documents = []
            
            # Open and read the JSONL file containing the PubMed documents
            with open(source_file, 'r') as json_file:
                for line in json_file:
                    data = json.loads(line)
                    documents.append(data)
                    
                    if len(documents) == 1000: # Bulk loading 1000 docs
                        collection.insert_many(documents)
                        documents = []  # Clear the list after insertion

            # Insert any remaining documents
            if documents:
                collection.insert_many(documents)

print("Indexing complete")

100%|██████████| 100/100 [21:06<00:00, 12.66s/it]

Indexing complete


In [31]:
doc_count = collection.count_documents({})
print(f"Es gibt {doc_count} Dokumente in der Sammlung.")

Es gibt 1795307 Dokumente in der Sammlung.


In [33]:
from pathlib import Path
import os
import json

# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client['RAG']
collection = db['PubMedEmbedded']
# Define the path for the source directory
source_directory = Path('C:/Users/linus/big_data/pubmed/test_chunk_embedded/')


def search_TF_IDF(query):
    results = collection.find({"$text": {"$search": query}}).limit(100)
    return results

pmid_liste = search_TF_IDF("List signaling molecules (ligands) that interact with the receptor EGFR?")
for i in pmid_liste:
    print(i['PMID'])

OperationFailure: text index required for $text query, full error: {'ok': 0.0, 'errmsg': 'text index required for $text query', 'code': 27, 'codeName': 'IndexNotFound'}

In [2]:
from Embedding import TextEmbedder
embedder = TextEmbedder()

In [28]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

collection = db['PubMedEmbedded']

def find_nearest_embeddings(query_embedding, embeddings, n=10):
    # Initialisieren Sie NearestNeighbors
    nearest_neighbors = NearestNeighbors(n_neighbors=n, algorithm='auto').fit(embeddings)
    # Finden Sie die n nächsten Nachbarn des Suchtext-Embeddings
    distances, indices = nearest_neighbors.kneighbors([query_embedding])
    return indices[0], distances[0]

def search_and_retrieve_embeddings(query):
    # Textsuche durchführen und gleichzeitig die Embeddings abrufen
    results = collection.find(
        {"$text": {"$search": query}},
        {'PMID': 1, 'embeddings': 1, '_id': 0}
    ).limit(1000)
    
    # Extrahieren Sie PMIDs und Embeddings aus den Ergebnissen
    pmids = []
    embeddings = []
    for doc in results:
        pmids.append(doc['PMID'])
        embeddings.append(doc['embeddings'])

    # Konvertieren Sie die Embeddings-Liste in ein NumPy-Array
    embeddings_array = np.array(embeddings)

    return pmids, embeddings_array

# Suchtext-Embedding generieren
such_text = "List signaling molecules (ligands) that interact with the receptor EGFR?"
pmids, embedding_matrix = search_and_retrieve_embeddings(such_text)
suchtext_embedding = embedder.embed(such_text)

# Finde die nächsten 10 Embeddings
indices, distances = find_nearest_embeddings(suchtext_embedding, embedding_matrix)

# Ergebnisse anzeigen
for i, index in enumerate(indices):
    print(f"{i + 1}: PMID = {pmids[index]}, Distanz = {distances[i]}")

1: PMID = 219769, Distanz = 4.967927935519385
2: PMID = 1813, Distanz = 4.968760284334843
3: PMID = 233131, Distanz = 4.979666667735479
4: PMID = 200939, Distanz = 4.981703585378111
5: PMID = 168935, Distanz = 4.984101335649784
6: PMID = 268625, Distanz = 4.987606585279695
7: PMID = 189643, Distanz = 5.0054406655705765
8: PMID = 200712, Distanz = 5.01118222034559
9: PMID = 95806, Distanz = 5.036777312349309
10: PMID = 175945, Distanz = 5.0384221205776685


In [ ]:
# save all PMIDs in list

